In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr2_example').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [11]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [10]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [12]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [13]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [14]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'], outputCol='features')

In [15]:
output = assembler.transform(data)

In [16]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [17]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [18]:
final_data = output.select('features', 'Yearly Amount Spent')

In [19]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [21]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [22]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                357|
|   mean|   499.552635176818|
| stddev|  80.61347235518743|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [23]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [24]:
lr_model = lr.fit(train_data)

In [25]:
test_results = lr_model.evaluate(test_data)

In [26]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -4.731679337961907|
|-12.842724590992873|
| 12.133960778666506|
|  22.45658331960226|
|  19.45135533197464|
| -8.303820706657575|
| -4.173256353540864|
|-15.717067413663301|
|  8.934190934339142|
|-0.1707929757800457|
|-1.3263839003831777|
|-17.069849694353024|
|-3.7146915396032227|
|  5.337842815586555|
| 0.3377034611443719|
| 17.284307731566514|
|  6.325327114521201|
|  17.19058420520099|
| 23.907078772291527|
|   -6.1803977064219|
+-------------------+
only showing top 20 rows



In [27]:
test_results.rootMeanSquaredError

10.564919124304222

In [28]:
test_results.r2

0.9806665096247402

In [29]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [30]:
unlabeled_data = test_data.select('features')

In [31]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[31.0662181616375...|
|[31.1695067987115...|
|[31.2834474760581...|
|[31.3123495994443...|
|[31.4474464941278...|
|[31.5147378578019...|
|[31.8164283341993...|
|[31.8512531286083...|
|[31.8627411090001...|
|[31.9120759292006...|
|[31.9563005605233...|
|[31.9673209478824...|
|[31.9764800614612...|
|[32.0047530203648...|
|[32.0180740106320...|
|[32.0478009788678...|
|[32.0478146331398...|
|[32.0498393904573...|
|[32.0883806304482...|
+--------------------+
only showing top 20 rows



In [32]:
predictions = lr_model.transform(unlabeled_data)

In [34]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...| 287.2029250578764|
|[31.0662181616375...| 461.7760177986672|
|[31.1695067987115...| 415.2225700236263|
|[31.2834474760581...| 569.3245061060652|
|[31.3123495994443...|  444.140062695966|
|[31.4474464941278...| 426.9065628018816|
|[31.5147378578019...| 493.9857443500023|
|[31.8164283341993...| 516.8395589173197|
|[31.8512531286083...|464.05805573245925|
|[31.8627411090001...| 556.4689341498267|
|[31.9120759292006...| 388.8611002060909|
|[31.9563005605233...| 564.1957814415518|
|[31.9673209478824...| 449.4645327792555|
|[31.9764800614612...|325.25660321851365|
|[32.0047530203648...|463.40827765948507|
|[32.0180740106320...| 340.4988030137488|
|[32.0478009788678...|507.12524407157525|
|[32.0478146331398...| 480.1989735536424|
|[32.0498393904573...|454.81227810192377|
|[32.0883806304482...|   518.34626409458|
+--------------------+------------

In [35]:
predictions.describe().show()

+-------+------------------+
|summary|        prediction|
+-------+------------------+
|  count|               143|
|   mean|496.01442426076034|
| stddev| 74.97802061667097|
|    min| 287.2029250578764|
|    max| 692.5877154521736|
+-------+------------------+

